In [ ]:
import pandas as pd
import sqlite3
import ast

conn = sqlite3.connect("statsbomb_euro2020.db")

events = pd.read_sql_query("SELECT match_id, team, team_id, type, location FROM events;", conn)
matches = pd.read_sql_query("SELECT match_id, home_team, away_team FROM matches;", conn)

def parse_location(val):
    if val is None:
        return None
    try:
        loc = ast.literal_eval(val)
        return loc if isinstance(loc, list) and len(loc) == 2 else None
    except:
        return None

events["location"] = events["location"].apply(parse_location)

passes = events[events["type"] == "Pass"].copy()

passes = pd.merge(passes, matches, on="match_id", how="left")

def is_final_third(row):
    loc = row["location"]
    if not loc:
        return False
    x = loc[0]
    if row["team"] == row["home_team"]:
        return x > 80
    elif row["team"] == row["away_team"]:
        return x < 40
    return False

passes["in_final_third"] = passes.apply(is_final_third, axis=1)

final_third_counts = (
    passes[passes["in_final_third"]]
    .groupby(["match_id", "team_id", "team"])
    .size()
    .reset_index(name="final_third_passes")
)

field_tilt_results = []
for match_id in final_third_counts["match_id"].unique():
    match_df = final_third_counts[final_third_counts["match_id"] == match_id]
    if len(match_df) == 2:
        team1 = match_df.iloc[0]
        team2 = match_df.iloc[1]
        total = team1["final_third_passes"] + team2["final_third_passes"]
        if total > 0:
            field_tilt_results.append({
                "match_id": match_id,
                "team": team1["team"],
                "team_id": team1["team_id"],
                "final_third_passes": team1["final_third_passes"],
                "field_tilt": round(team1["final_third_passes"] / total * 100, 2)
            })
            field_tilt_results.append({
                "match_id": match_id,
                "team": team2["team"],
                "team_id": team2["team_id"],
                "final_third_passes": team2["final_third_passes"],
                "field_tilt": round(team2["final_third_passes"] / total * 100, 2)
            })

field_tilt_df = pd.DataFrame(field_tilt_results)

field_tilt_df.to_csv("field_tilt_per_match.csv", index=False, encoding="utf-8-sig")
print("saved as field_tilt_per_match.csv")

avg_field_tilt = (
    field_tilt_df.groupby("team", as_index=False)["field_tilt"]
    .mean()
    .sort_values(by="field_tilt", ascending=False)
)
avg_field_tilt.to_csv("field_tilt_team_average.csv", index=False, encoding="utf-8-sig")
print("saved as field_tilt_team_average.csv")

print("\ntop 10 Field Tilt teams")
print(avg_field_tilt.head(10))


saved as field_tilt_per_match.csv
saved as field_tilt_team_average.csv

top 10 Field Tilt teams
           team  field_tilt
18        Spain   76.861667
8       Germany   64.490000
7        France   56.262500
0       Austria   56.170000
14     Portugal   55.037500
4       Denmark   52.515000
1       Belgium   50.656000
11  Netherlands   50.047500
10        Italy   49.938571
5       England   49.935714
